![](http://)# **RSNA Pneumonia Detection Challenge**

## **Overview**


![alt text](https://i.pinimg.com/564x/ff/cd/c4/ffcdc4d74eed036d029a84c381604a10.jpg)

## **Symptoms to detect Pneumonia**

The Diagnosis of pneumonia on CXR( is complicated because of a number of other nditions in the lungsuch as uid overload (pulmonary edema), bleeding, volume loss (atelectasis or collapse), lung cancer, or post-radiation or surgical changes.

## **1-Exploring the data**

In [ ]:
import glob, pylab, pandas as pd
import pydicom, numpy as np

import os
import csv
import random
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt

!ls ../input

In [ ]:
df = pd.read_csv('../input/stage_1_train_labels.csv')
print(df.iloc[0])

In [ ]:
print(df.iloc[8])

## **Data Summary**

**Stage 1 Images** - stage_1_train_images.zip and stage_1_test_images.zip
images for the current stage. Filenames are also patient names.

**Stage 1 Labels** - stage_1_train_labels.csv and Stage 1 Sample Submission stage_1_sample_submission.csv
Which provides the IDs for the test set, as well as a sample of what your submission should look like

**Stage 1 Detailed Info** - stage_1_detailed_class_info.csv
contains detailed information about the positive and negative classes in the training set, and may be used to build more nuanced models.

## **File descriptions**

**stage_1_train.csv** - the training set. Contains patientIds and bounding box / target information.

**stage_1_sample_submission.csv** - a sample submission file in the correct format.

**stage_1_detailed_class_info.csv** - provides detailed information about the type of positive or negative class for each image.

## **Data fields**
**patientId _**- A patientId. Each patientId corresponds to a unique image.

**x_ **- the upper-left x coordinate of the bounding box.

**y_ **- the upper-left y coordinate of the bounding box.

**width_** - the width of the bounding box.

**height_** - the height of the bounding box.

**Target_** - the binary Target, indicating whether this sample has evidence of pneumonia.

In [ ]:
from os import listdir
from os.path import isfile, join


det_class_path = '../input/stage_1_detailed_class_info.csv'
bbox_path = '../input/stage_1_train_labels.csv'
dicom_dir = '../input/stage_1_train_images/'

train_images_dir = '../input/stage_1_train_images/'
train_images = [f for f in listdir(train_images_dir) if isfile(join(train_images_dir, f))]
test_images_dir = '../input/stage_1_test_images/'
test_images = [f for f in listdir(test_images_dir) if isfile(join(test_images_dir, f))]

print('Number of train images:', len(train_images))
print('Number of test images:', len(test_images))

## **DICOM files:**

In [ ]:
patientId = df['patientId'][4]
dcm_file = '../input/stage_1_train_images/%s.dcm' % patientId
dcm_data = pydicom.read_file(dcm_file)

print(dcm_data)

In [ ]:
patientId2 = df['patientId'][55]
dcm_file2 = '../input/stage_1_train_images/%s.dcm' % patientId2
dcm_data2 = pydicom.read_file(dcm_file2)

print(dcm_data2)

In [ ]:
im = dcm_data.pixel_array
print(type(im))
print(im.dtype)
print(im.shape)

* high bit-depth original images have been rescaled to 8-bit encoding (256 grayscales)
* Thel image matrices (typically acquired at >2000 x 2000) have been resized to the shape of 1024 x 1024

In [ ]:
pylab.imshow(im, cmap=pylab.cm.gist_gray)

In [ ]:
im2 = dcm_data2.pixel_array
pylab.imshow(im2, cmap=pylab.cm.gist_gray)

## **CSV into a data structure with unique entries (the patient ID):**

Any patient may  have many boxes if there are several different suspicious areas of pneumonia. 

In [ ]:
def parse_data(df):
    """
      parsed = {
        
        'patientId-00': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        },
        'patientId-01': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        }, ...

      }

    """
    # Define lambda to extract coords in list [y, x, height, width]
    extract_box = lambda row: [row['y'], row['x'], row['height'], row['width']]

    parsed = {}
    for n, row in df.iterrows():
        pid = row['patientId']
        if pid not in parsed:
            parsed[pid] = {
                'dicom': '../input/stage_1_train_images/%s.dcm' % pid,
                'label': row['Target'],
                'boxes': []}

        # Add box if opacity is present
        if parsed[pid]['label'] == 1:
            parsed[pid]['boxes'].append(extract_box(row))

    return parsed



parsed = parse_data(df)



print(parsed['00436515-870c-4b36-a041-de91049b9ab4'])


## **Overlay color boxes on the original grayscale DICOM files:****

In [ ]:
def draw(data):
    #Open DICOM file
    d = pydicom.read_file(data['dicom'])
    im = d.pixel_array

    #Convert from single-channel grayscale to 3-channel RGB
    im = np.stack([im] * 3, axis=2)

    #Add boxes with random color if present
    for box in data['boxes']:
        rgb = np.floor(np.random.rand(3) * 256).astype('int')
        im = overlay_box(im=im, box=box, rgb=rgb, stroke=6)

    pylab.imshow(im, cmap=pylab.cm.gist_gray)
    
    

def overlay_box(im, box, rgb, stroke=1):
    #Convert coordinates to integers
    box = [int(b) for b in box]
    
    #Extract coordinates
    y1, x1, height, width = box
    y2 = y1 + height
    x2 = x1 + width

    im[y1:y1 + stroke, x1:x2] = rgb
    im[y2:y2 + stroke, x1:x2] = rgb
    im[y1:y2, x1:x1 + stroke] = rgb
    im[y1:y2, x2:x2 + stroke] = rgb

    return im



draw(parsed['00436515-870c-4b36-a041-de91049b9ab4'])

## **Exploring Detailed Labels**

In addition to the binary classification (the presence or absence of pneumonia), each bounding box without pneumonia is further categorized into normal or no lung opacity / not normal (abnormality on the image)

In [ ]:
df_detailed = pd.read_csv('../input/stage_1_detailed_class_info.csv')
print(df_detailed.iloc[6])
print(df_detailed.iloc[80])

**Model**

In [ ]:
# empty dictionary
nodule_locations = {}
# load table
with open(os.path.join('../input/stage_1_train_labels.csv'), mode='r') as infile:
    reader = csv.reader(infile)
    # skip header
    next(reader, None)

    for rows in reader:
        filename = rows[0]
        location = rows[1:5]
        nodule = rows[5]
        # if row contains a nodule add label to dictionary
        # which contains a list of nodule locations per filename
        if nodule == '1':
            # convert string to float to int
            location = [int(float(i)) for i in location]
            # save nodule location in dictionary
            if filename in nodule_locations:
                nodule_locations[filename].append(location)
            else:
                nodule_locations[filename] = [location]


In [ ]:
folder = '../input/stage_1_train_images'
filenames = os.listdir(folder)
random.shuffle(filenames)
# split into train and validation filenames
n_valid_samples = 2000
train_filenames = filenames[n_valid_samples:]
valid_filenames = filenames[:n_valid_samples]
print('n train samples', len(train_filenames))
print('n valid samples', len(valid_filenames))
n_train_samples = len(filenames) - n_valid_samples

In [ ]:
class generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, nodule_locations=None, batch_size=32, image_size=256, shuffle=True, predict=False):
        self.folder = folder
        self.filenames = filenames
        self.nodule_locations = nodule_locations
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.predict = predict
        self.on_epoch_end()
        
    def __load__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # create empty mask
        msk = np.zeros(img.shape)
        # get filename without extension
        filename = filename.split('.')[0]
        # if image contains nodules
        if filename in nodule_locations:
            # loop through nodules
            for location in nodule_locations[filename]:
                # add 1's at the location of the nodule
                x, y, w, h = location
                msk[y:y+h, x:x+w] = 1
        # resize both image and mask
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        msk = resize(msk, (self.image_size, self.image_size), mode='reflect') > 0.5
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)
        return img, msk
    
    def __loadpredict__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # resize image
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img
        
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # predict mode: return images and filenames
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, filenames
        # train mode: return images and masks
        else:
            # load files
            items = [self.__load__(filename) for filename in filenames]
            # unzip images and masks
            imgs, msks = zip(*items)
            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
        
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            # return full batches only
            return int(len(self.filenames) / self.batch_size)

In [ ]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
    # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# define iou or jaccard loss function
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

# combine bce loss and iou loss
def iou_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)

# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

# create network and compiler
model = create_network(input_size=256, channels=32, n_blocks=2, depth=4)
model.compile(optimizer='adam',
              loss=iou_bce_loss,
              metrics=['accuracy', mean_iou])

# cosine learning rate annealing
def cosine_annealing(x):
    lr = 0.001
    epochs = 25
    return lr*(np.cos(np.pi*x/epochs)+1.)/2
learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

# create train and validation generators
folder = '../input/stage_1_train_images'
train_gen = generator(folder, train_filenames, nodule_locations, batch_size=32, image_size=256, shuffle=True, predict=False)
valid_gen = generator(folder, valid_filenames, nodule_locations, batch_size=32, image_size=256, shuffle=False, predict=False)

history = model.fit_generator(train_gen, validation_data=valid_gen, callbacks=[learning_rate], epochs=2, shuffle=True)

In [ ]:
from sklearn import feature_selection, linear_model, metrics, preprocessing


folder = '../input/stage_1_test_images/'
filenames = os.listdir(folder)
#random.shuffle(filenames)





In [ ]:
n_y_samples = 1 #must be at least 500 (it is currently used to test 1 image only)
test_X = filenames[999:]  #must be: test_X = filenames[n_y_samples:]
test_Y = filenames[:n_y_samples]
print('n test_x', len(test_X))
print('n test_y', len(test_Y))
n_x_samples = len(filenames) - n_y_samples

In [ ]:
#should return a shape of (1, 32, 32, 3)  ???

test_x_gen = generator(folder, test_X, nodule_locations, batch_size=32, image_size=256, shuffle=True, predict=False)
test_y_gen = generator(folder, test_Y, nodule_locations, batch_size=32, image_size=256, shuffle=True, predict=False)

#print (test_x_gen)
#print (test_y_gen)
    


In [ ]:
#score = model.evaluate(np.expand_dims(test_x_gen, axis=3), test_y_gen, batch_size=32)
#print (score)

#model.predict(test_x_gen)
              
#metrics.accuracy_score(np.array(test_y_gen), model.predict(np.array(test_x_gen)))
